# Vanilla Net

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def clean_train_data(news_df, market_df):
    '''Clean and preprocess the news and market data for training.
    
    Parameters
    ----------
    news_df : dataframe
        See https://www.kaggle.com/c/two-sigma-financial-news/data for full description of the dataframe.
    market_df : dataframe
        See https://www.kaggle.com/c/two-sigma-financial-news/data for full description of the dataframe.
    
    Returns
    -------
    dataframe 
        Cleaned data ready to be fed to the model.
    
    '''
    # assetCode, time, volume, open, returnsOpenPrevMktres1, 
    # returnsOpenPrevMkres10, returnsOpenNextMktres10
    # sentimentNegative, sentimentNeutral, sentimentPositive
    cols = ['assetCode', 'time', 'volume', 'open', 'returnsOpenPrevMktres1', 
            'returnsOpenPrevMkres10', 'returnsOpenNextMktres10']
    cleaned_df = market_df.loc[cols]
    
    return None

#TODO: Add cleaned data specifications
#TODO: Define Returns
def train_model(train_df):
    '''Train the model using the given trianing data.
    
    Parameters
    ----------
    train_data : dataframe
        Cleaned data. (Specifications)
        
    Returns
    -------

    '''
    
    return None



## Get competition environment

In [4]:
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

Loading the data... This could take a minute.
Done!


## Get training data

In [5]:
(market_train_df, news_train_df) = env.get_training_data()

## Preprocess and clean the data

In [13]:
# Select relevant columns
cols = ['assetCode', 'time', 'volume', 'open', 'returnsOpenPrevMktres1', 
        'returnsOpenPrevMktres10', 'returnsOpenNextMktres10']
market_train_df = market_train_df.loc[:,cols]
market_train_df.dropna(inplace=True)

In [14]:
# Select wanted columns
cols = ['time', 'urgency','assetCodes', 'relevance', 'sentimentNegative', 'sentimentNeutral', 'sentimentPositive']
news_train_df = news_train_df.loc[:,cols]
news_train_df.dropna(inplace=True)

In [15]:
from itertools import chain

# Create new dataframe where each entry represents one time/asset pair
train_df = market_train_df.copy()

# Normalize time
train_df.loc[:, 'time'] = train_df.time.dt.normalize()
news_train_df.loc[:, 'time'] = news_train_df.time.dt.normalize()

# assetCodes from String to List
news_train_df['assetCodes'] = news_train_df['assetCodes'].str.findall(f"'([\w\./]+)'")

In [21]:
# assetCodes = list(chain(*news_train_df['assetCodes']))
assetCodes_expanded = list(chain(*news_train_df['assetCodes']))
assetCodes_index = news_train_df.index.repeat(news_train_df['assetCodes'].apply(len))

Int64Index([      0,       0,       0,       0,       1,       2,       2,
                  3,       3,       3,
            ...
            9328743, 9328744, 9328744, 9328745, 9328746, 9328747, 9328747,
            9328748, 9328748, 9328749],
           dtype='int64', length=18821885)


In [ ]:
clean_train_df = clean_data(news_train_df, market_train_df)

## Train the model

In [ ]:
train_model(train_df)

## Make predictions on test data

In [ ]:
days = env.get_prediction_days()

In [ ]:
import numpy as np
def make_random_predictions(predictions_df):
    predictions_df.confidenceValue = 2.0 * np.random.rand(len(predictions_df)) - 1.0

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    make_random_predictions(predictions_template_df)
    env.predict(predictions_template_df)
print('Done!')

In [ ]:
env.write_submission_file()